<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/DecisionTree/DecisionTree_TFidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1-import library
 
  إستيراد المكتبات

In [80]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from joblib import dump, load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


--------

2- import Datasets

استيراد ملفات البيانات


In [89]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")

------
3- shuffle  
بعثرة البيانات

In [90]:
df=shuffle(df)
df

,text,targe
14822,فاز فريق شباب أطلس خنيفرة على ضيفه شباب الريف ...,4
38927,أفرجت النيابة العامة بابتدائية صفرو صباح الجمع...,1
15800,اعترف بوشعيب المباركي اللاعب السابق بفريق الرج...,4
113,سفير الملحون تحدث خلال البرنامج عن بداياته الف...,0
7610,محمد اسليم أخبارنا المغربية بالفعل واستنادا لم...,2
...,...,...
7024,أخبارنا وافق الاتحاد الأوروبي يوم الجمعة على ت...,2
17713,توفي طبيب الأعصاب أوليفر ساكس الذي تخصص في درا...,0
39478,بنعبد الله وشباط يرعيان حوارا لتشكيل حكومة مع ...,1
33748,أعطى المدرب عبد الرحيم طالب موافقته على تمديد ...,4


------
4- clean data

تنظيف البيانات

In [91]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

367
0
text     True
targe    True
dtype: bool
52500
                                                    text  targe
14822  فاز فريق شباب اطلس خنيفره علي ضيفه شباب الريف ...      4
38927  افرجت النيابه العامه بابتدائيه صفرو صباح الجمع...      1
15800  اعترف بوشعيب المباركي الاعب السابق بفريق الرجا...      4
113    سفير الملحون تحدث خلال البرنامج عن بداياته الف...      0
7610   محمد اسليم اخبارنا المغربيه بالفعل واستنادا لم...      2
...                                                  ...    ...
7024   اخبارنا وافق الاتحاد الاوروبي يوم الجمعه علي ت...      2
17713  توفي طبيب الاعصاب اوليفر ساكس الذي تخص في دراس...      0
39478  بنعبد اله وشباط يرعيان حوارا لتشكيل حكومه مع ب...      1
33748  اعطي المدرب عبد الرحيم طالب موافقته علي تمديد ...      4
6078   اعلنت وزاره الداخليه انه في اطار المجهودات الا...      1

[52133 rows x 2 columns]


----
5-
Tokinaizer: TFidf

تحويل النصوص إلى أرقام عبر خوازرمية
TfidfVectorizer




In [93]:
y = clean_df['targe']
X = clean_df['text']
stopwords_list = stopwords.words('arabic')

tfidf = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7,token_pattern=r"(?u)\b\w\w+\b",stop_words=stopwords_list)

y1 = clean_df['targe']
X1 = clean_df['text']
X1= tfidf.fit_transform (X1) .toarray ()


----
6- accuracy calculation

عمل تابع لحساب الدقة

In [ ]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

---------------------
7- classifier:DecisionTreeClassifier

with Cross validation


التنصنيف عبر خوارزمية

DecisionTreeClassifier


استخدام خوارزمية 
Cross validation
لتوزيع عينات الاختبار والتأكد من عدم وجود 
Overfitting

In [94]:
from sklearn.model_selection import StratifiedKFold
from sklearn import tree

strtfdKFold = StratifiedKFold(n_splits=5)
kfold = strtfdKFold.split(X1, y1)
clf = tree.DecisionTreeClassifier(random_state=5)
result_DecisionTreeClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    model = clf.fit(X_train, y_train)
    y_pred=(model.predict(X_test))
    result_DecisionTreeClassifier.append(calculate_results(y_test, y_pred))


حساب متوسط النتائج للـ5 مراحل في cross validation 

In [95]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_DecisionTreeClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_DecisionTreeClassifier)
f1=f1/len(result_DecisionTreeClassifier)
precision=precision/len(result_DecisionTreeClassifier)
recall=recall/len(result_DecisionTreeClassifier)
# (result_DecisionTreeClassifier)
results_DecisionTreeClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_DecisionTreeClassifier

[{'accuracy': 84.94045040256954,
  'f1': 0.8495581834605119,
  'precision': 0.8497768376618365,
  'recall': 0.8494045040256953}]

-----------------------
-----------------------
-----------------------
8-Results

 النتائج

نتائج التدريب لكل قاعدة بيانات

In [79]:
print('DecisionTreeClassifier results in dataset 1:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 1:


[{'accuracy': 84.75231381904115,
  'f1': 0.8477014138491688,
  'precision': 0.8479472021812396,
  'recall': 0.8475231381904116}]

In [88]:
print('DecisionTreeClassifier results in dataset 2:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 2:


[{'accuracy': 86.4940590922711,
  'f1': 0.8650148783873235,
  'precision': 0.8652240244394017,
  'recall': 0.8649405909227109}]

In [96]:
print('DecisionTreeClassifier results in dataset 3:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 3:


[{'accuracy': 84.94045040256954,
  'f1': 0.8495581834605119,
  'precision': 0.8497768376618365,
  'recall': 0.8494045040256953}]

In [71]:
print('DecisionTreeClassifier results in dataset 4:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 4:


[{'accuracy': 84.7669488926263,
  'f1': 0.8478120876570386,
  'precision': 0.8480099659559468,
  'recall': 0.8476694889262631}]

In [108]:
Final_Result=[{'accuracy': 84.75231381904115,
  'f1': 0.8477014138491688,
  'precision': 0.8479472021812396,
  'recall': 0.8475231381904116},
  {'accuracy': 86.4940590922711,
  'f1': 0.8650148783873235,
  'precision': 0.8652240244394017,
  'recall': 0.8649405909227109},
  {'accuracy': 84.94045040256954,
  'f1': 0.8495581834605119,
  'precision': 0.8497768376618365,
  'recall': 0.8494045040256953},
  {'accuracy': 84.7669488926263,
  'f1': 0.8478120876570386,
  'precision': 0.8480099659559468,
  'recall': 0.8476694889262631}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]

print('Final_Result:')
print(Final_Result)

Final_Result:
[{'accuracy': 85.23844305162703, 'f1': 0.8525216408385107, 'precision': 0.8527395075596063, 'recall': 0.8523844305162702}]
